In [1]:
MODEL_PATH = './FinalProjData/model/7_class_emotion_detector_with_transformations_with_oversampling.h5'

In [8]:
import cv2
from keras.models import load_model
import numpy as np

emotions2int = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
int2emotions = {v:k for k,v in emotions2int.items()}
model = load_model(MODEL_PATH)

classifier = cv2.CascadeClassifier('./FinalProjData/Haarcascades/haarcascade_frontalface_default.xml')

def detect_face(frame):
    faces=classifier.detectMultiScale(frame,1.3,4)
    if len(faces) == 0:
        return frame
    for x,y,w,h in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(172,42,251),2)
        face = frame[y:y+h,x:x+w]
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        face = cv2.resize(face,(48,48))
        face = face.reshape(1,48,48,1)
        cv2.putText(frame,text=int2emotions[np.argmax(model.predict(face))],
                    org=(x,y-15),fontFace=cv2.FONT_HERSHEY_SIMPLEX,fontScale=1,color=(106,40,243),thickness=2)
    return frame

cap = cv2.VideoCapture(0)

while 1:
    ret,frame= cap.read()
    if ret==True:
        cv2.imshow('emotion_detector',detect_face(frame))
        if cv2.waitKey(1)==27:
            break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


KeyboardInterrupt: 

In [11]:
# Iterate on images and save them after detection

from pathlib import Path

source_images = Path('./RealFaces').glob("*.jpg")
raz_images = Path("./RealFaces/Raz").glob("*.jpg")

In [12]:

for image in source_images:
    frame= cv2.imread(image.as_posix())
    cv2.imwrite((image.parent / f"{image.stem}_detected.jpg").as_posix(), detect_face(frame))


1/1 [==============================] - 0s 33ms/step


In [13]:

for image in raz_images:
    frame= cv2.imread(image.as_posix())
    cv2.imwrite((image.parent / f"{image.stem}_detected.jpg").as_posix(), detect_face(frame))


1/1 [==============================] - 0s 22ms/step


In [9]:
raz_movie = "RealFaces/raz/IMG_0789.MOV"

cap = cv2.VideoCapture(raz_movie)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)

result = cv2.VideoWriter('filename.avi',
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    result.write(detect_face(frame))

cap.release()
result.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 27ms/step
Can't receive frame (stream end?). Exiting ...
